# ETAPA 5: Rodar NER + RE - PREPARADO PARA EXECUTAR O EXPERIMENTO 04 DE IDEA-C2 - TESE DE DOUTORADO

## Objetivo: Submeter um conjunto de sentenças de textos, baseado no minimundo de CROMO-MOS, ao modelo pré-treinado no domínio e extrair as NER e RE

## Entrada: .../input/inputs.txt
# Pré-requisitos:
### Modelo NER treinado: NER_MODEL_PATH = ".../NER/outputs/model-best/"
### Modelo RE treinado: RE_MODEL_PATH = ".../RE/rel_component/training/model-best"
### Recuperar o MAP_LABELS utilizado no RE
## Etapas:
### Etapa 1: Montar o Drive
### Etapa 2: Configurações básicas do Spacy
### Etapa 3: Definir parâmetros de extração
### Etapa 4: Submeter o texto ao modelo de NER e RE já treinado no domínio
## Saída dos arquivos gerados: trecho.txt, terms.txt e relations.txt



In [ ]:
#***************************************************************************************************************************
# Etapa 1: Montar o Drive
#***************************************************************************************************************************
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#***************************************************************************************************************************
# Etapa 2: Configurações básicas do Spacy
# OBS: Estão sendo utilizadas funções externas dos arquivos: rel_pipe.py, rel_model.py e auxiliares.py
#***************************************************************************************************************************
# Instalando as bibliotecas essenciais do Spacy
!pip install -U spacy
!python -m spacy download pt_core_news_sm
!python -m spacy download pt_core_news_md
!pip install spacy-transformers
!pip install unidecode

# Realizando os imports
import spacy
import pandas as pd
import sys
import os
import random
import typer
from pathlib import Path
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example
sys.path.append('/content/drive/MyDrive/Doutorado/IDEA/RE/rel_component')
from scripts.rel_pipe import make_relation_extractor, score_relations
from scripts.rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors
sys.path.append('/content/drive/MyDrive/Doutorado')
from IDEA.auxiliares import *
from IDEA.singularize import *
import unidecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 11.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 7.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.8/795.8 kB 57.5 MB/s eta 0:00:00
   ━

In [ ]:
##***************************************************************************************************************************
# Etapa 3: Definir parâmetros de extração
#***************************************************************************************************************************

# Local do modelo NER ajustado ao domínio
NER_MODEL_PATH = "/content/drive/MyDrive/Doutorado/IDEA/NER/outputs/model-last-md-10-05-2024-corpus_completo_area_de_sobrevoo/"

# Local do modelo RE ajustado ao domínio
RE_MODEL_PATH = "/content/drive/MyDrive/Doutorado/IDEA/RE2/rel_component/training2/model-last-sm-10-05-2024-corpus_completo_area_de_sobrevoo/"


LIMIAR = 0.4
#QUEBRA_SENTS = True
QUEBRA_SENTS = False


# Apontar na constante LABELS qual o MAP_LABELS associado
LABELS = 'IDEA'

# Esse MAP_LABELS é definido no notebook IDEA-ConverterDoccanoSpacy

MAP_LABELS_IDEA = {
    "composed_of": "composed_of",
    "instance_of": "instance_of",
    "applied_to": "applied_to",
    "occurs_in": "occurs_in",
    "co-referenced": "co-referenced",
    "responsible_for": "responsible_for",
    "capacity_of": "capacity_of",
    "type_of": "type_of",
    "defined_by": "defined_by",
    "capacity_of": "capacity_of",
    "equivalent_to": "equivalent_to",
    "associated_with": "associated_with"
}

MAP_LABELS = MAP_LABELS_IDEA

print("Configuracoes:")
Sent = 'Sim' if QUEBRA_SENTS else 'Nao'
print(f"   {LABELS} -- Limiar = {LIMIAR} -- Quebra em sentenças? {Sent}")
###########################################################################

Configuracoes:
   IDEA -- Limiar = 0.4 -- Quebra em sentenças? Nao


##Etapa: 4: Submeter o texto ao modelo de NER e RE já treinado no domínio

### Passo 1: Obter os modelos de NER e RE treinados no domínio (nlp e nlp2)
### Passo 2: Obter dados de entrada (textos para avaliar): inputs.txt
### Passo 3: Submeter os textos de "<<arquivo-trecho-doutrina.txt>> ao modelo NER treinado (IDEA-C2-Model-NER)
### Passo 4: Inferir as Entidades e armazenar em uma estrutura de dados DICT (dim_ents)
### Passo 5: Com base nas Entidades obtidas, inferir as relações no modelo RE treinado (IDEA-C2-Model-RE) e armazenar o seu conteúdo no DICT (dim_rels)
### Passo 6: Recuperar o trecho submetido aos modelos (IDEA-C2-Model - NER e RE), os resultados das inferências (dim_ents e dim_rels) e salvar, respectivamente, nos arquivos de saída: trecho.txt, terms.txt e relations.txt

### Passo 1: Obter os modelos de NER e RE treinados no domínio (nlp e nlp2)


In [ ]:
import spacy
import pandas as pd
#***************************************************************************************************************************
# Passo 1: Obter os modelos de NER e RE treinados no domínio (nlp e nlp2)
#***************************************************************************************************************************
# Modelo NER
nlp = spacy.load(NER_MODEL_PATH)
nlp.add_pipe('sentencizer')

# Modelo RE
nlp2 = spacy.load(RE_MODEL_PATH)
nlp2.add_pipe('sentencizer')

# sem fine tuning (bertimbau)
nlp3 = spacy.load("pt_core_news_sm")
nlp3.add_pipe('sentencizer')

# sem fine tuning (bertimbau)
nlp4 = spacy.load("pt_core_news_md")
nlp4.add_pipe('sentencizer')

# sem fine tuning (bert multiligual)
# nlp4 = spacy.load("xx_ent_wiki_sm")
# nlp4.add_pipe('sentencizer')

/usr/local/lib/python3.12/dist-packages/spacy/util.py:969: UserWarning: [W095] Model 'pt_pipeline' (0.0.0) was trained with spaCy v3.7.4 and may not be 100% compatible with the current version (3.8.11). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/spacy/util.py:969: UserWarning: [W095] Model 'pt_pipeline' (0.0.0) was trained with spaCy v3.7.4 and may not be 100% compatible with the current version (3.8.11). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Passo 2: Obter dados de entrada: "<<CT>>"


In [ ]:
#***************************************************************************************************************************
# Passo 2: Submeter o texto de entrada aos modelos NER e RE treinados para inferir as Entidades e Relações
#***************************************************************************************************************************
PASTA_ARQUIVO_INPUT = "/content/drive/MyDrive/Doutorado/IDEA/input/"
NOME_ARQUIVO_INPUT = PASTA_ARQUIVO_INPUT + "exp4_minimundo.txt"

# read -> para arquivos simples (ex: senhas, tokens, informações únicas)
with open(NOME_ARQUIVO_INPUT, "r") as arquivo:
    text = arquivo.readlines()
print(text)

['Uma Operação Militar (Military OPeration) possui diferentes subtipos, como Operações Básicas e Complementares (Basic and Complentary Operations). As Operações básicas são operações que, por si só, podem atingir os objetivos determinados por uma autoridade em uma situação de guerra ou não-guerra. As operações complementares, por outro lado, destinam-se a ampliar, melhorar e/ou complementar as operações básicas [35]. Uma Operação Ofensiva (Offensive Operation) é uma das Operações Básicas que se inicia e se desdobra em quatro operações distintas: (i) Reunião de Preparação (Assembly Area), quando o comandante se reúne com seus subordinados para trocar informações e transmitir ordens; (ii) Marcha para o Combate (Movement to Contact), quando as forças participantes marcham em direção às forças inimigas; (iii) Ataque Coordenado (Organized Offensive), momento em que ocorre a ação de atacar as forças inimigas; e (iv) Aproveitamento do êxito e Perseguição (Exploitation and Follow Up), uma vez 

In [ ]:
def obter_ct(arquivo):
  #***************************************************************************************************************************
  # Passo 2: Submeter o texto de entrada aos modelos NER e RE treinados para inferir as Entidades e Relações
  #***************************************************************************************************************************
  PASTA_ARQUIVO_INPUT = "/content/drive/MyDrive/Doutorado/IDEA/input/"
  NOME_ARQUIVO_INPUT = PASTA_ARQUIVO_INPUT + arquivo

  # read -> para arquivos simples (ex: senhas, tokens, informações únicas)
  with open(NOME_ARQUIVO_INPUT, "r") as arquivo:
      text = arquivo.readlines()
  print(text)
  return text

In [ ]:
ct=obter_ct('exp4_minimundo.txt')

['Uma Operação Militar (Military OPeration) possui diferentes subtipos, como Operações Básicas e Complementares (Basic and Complentary Operations). As Operações básicas são operações que, por si só, podem atingir os objetivos determinados por uma autoridade em uma situação de guerra ou não-guerra. As operações complementares, por outro lado, destinam-se a ampliar, melhorar e/ou complementar as operações básicas [35]. Uma Operação Ofensiva (Offensive Operation) é uma das Operações Básicas que se inicia e se desdobra em quatro operações distintas: (i) Reunião de Preparação (Assembly Area), quando o comandante se reúne com seus subordinados para trocar informações e transmitir ordens; (ii) Marcha para o Combate (Movement to Contact), quando as forças participantes marcham em direção às forças inimigas; (iii) Ataque Coordenado (Organized Offensive), momento em que ocorre a ação de atacar as forças inimigas; e (iv) Aproveitamento do êxito e Perseguição (Exploitation and Follow Up), uma vez 

#Passos 3, 4 e 5:
#Passo 3: Submeter os textos de "<<arquivo-trecho-doutrina.txt>> ao modelo NER treinado (IDEA-C2-Model-NER)
# Passo 4: Inferir as Entidades e armazenar em uma estrutura de dados DICT (dim_ents)
#Passo 5: Com base nas Entidades obtidas, inferir as relações no modelo RE treinado (IDEA-C2-Model-RE) e armazenar o seu conteúdo no DICT (dim_rels)

In [ ]:
def executar_modelo_ner_e_re(text):
  id_ent_cont= 0 # variável que conta as entidades nomeadas encontradas
  id_rel_cont = 0 # variável que conta as relações encontradas

  dim_ents = [] # vetor que armazena as entidades encontradas
  fato_ents = []

  fato_rels = []
  dim_rels = []

  ent_id_start = {}  # Dicionario para identificar (depois, no armazenamento das relacoes) o id da entidade pelo seu start

  for cont_docum, doc in enumerate(nlp.pipe(text, disable=["tagger"])):
    #print('\nTexto:', doc.text)
    #print('\nEntidades:')
    for e in doc.ents:
      nr_ent = recupera_entidade(dim_ents, e.text, e.label_)
      #print(f'   {e.start_char}\t{e.text} [{e.label_}]')
      if nr_ent == -1:
        # print(e.text, "não encontrada. Inserindo...")
        # print(id_ent_cont, e.text, e.label_)
        dim_ents.append([id_ent_cont, e.text, e.label_])
        #fato_ents.append([id_doc, id_ent_cont, 1])
      else:
        print("já tinha")
        #fato_ents.append([id_doc, nr_ent, 1])
      ent_id_start[e.start_char] = id_ent_cont  # So posso fazer isso porque modelo nao aceita overlap
      id_ent_cont += 1

    #print('\nTodas entidades já foram processadas...')
    #print(ent_id_start)

    #print('dim_ents:', dim_ents)

  # #***************************************************************************************************************************
  # # Passo 5: Com base nas NER obtidas, identificar as relações no modelo RE treinado
  # #***************************************************************************************************************************
    for name, proc in nlp2.pipeline:
      doc = proc(doc)
      print('\nRelações:')
      if (QUEBRA_SENTS):
        # Aqui, dividimos o parágrafo em frases e aplicamos a extração de relação
        # para cada par de entidades encontradas em cada sentença..
        for value, rel_dict in doc._.rel.items():
          for sent in doc.sents:
            for e in sent.ents:
              for b in sent.ents:
                #print('>> e.start_char:', e.start_char, 'b.start_char:', b.start_char)
                id_ent_e = ent_id_start[e.start_char]  # Recuperando id da entidade head pelo seu start
                id_ent_b = ent_id_start[b.start_char]  # Recuperando id da entidade tail pelo seu start
                if e.start == value[0] and b.start == value[1]:
                  for label in MAP_LABELS.values():
                    #print("rel_dict[label]", rel_dict[label])
                    #if rel_dict[label] >= LIMIAR:
                    if rel_dict[label] > 0.015:
                      #print(f"     {e.text} --{label}-- {b.text} [{rel_dict[label]:.2%}]")
                      #dim_rels.append([id_rel_cont, label])
                      dim_rels.append([id_rel_cont, e.text, label,b.text])
                      #fato_rels.append([id_doc, id_ent_e, id_ent_b, id_rel_cont])
                      id_rel_cont += 1
      else:
        # Sem quebrar em sentenças
        for value, rel_dict in doc._.rel.items():
          for e in doc.ents:
            for b in doc.ents:
              #print('>> e.start_char:', e.start_char, 'b.start_char:', b.start_char)
              id_ent_e = ent_id_start[e.start_char]  # Recuperando id da entidade head pelo seu start
              id_ent_b = ent_id_start[b.start_char]  # Recuperando id da entidade tail pelo seu start
              if e.start == value[0] and b.start == value[1]:
                for label in MAP_LABELS.values():
                  #if rel_dict[label] > 0.0005:
                  if rel_dict[label] > 0.015:
                    print(f"     {e.text} --{label}-- {b.text} [{rel_dict[label]:.2%}]")
                    dim_rels.append([id_rel_cont, e.text, label,b.text])
                    #fato_rels.append([id_doc, id_ent_e, id_ent_b, id_rel_cont])
                    id_rel_cont += 1
      print('\n')
      return dim_ents, dim_rels

In [ ]:
ents, rels = executar_modelo_ner_e_re(ct)

	Verificando: Operação Militar entity
	Verificando: autoridade entity
	Verificando: guerra entity
	Verificando: Operação Ofensiva entity
	Verificando: Preparação entity
	Verificando: comandante entity
	Verificando: Marcha entity
	Verificando: Ataque entity
	Verificando: ação entity
	Verificando: Perseguição entity
	Verificando: ataque entity
	Verificando: inimigo entity
	Verificando: operação entity
	Verificando: Controle de Tráfego entity
	Verificando: Reconhecimento entity
	Verificando: operação entity
operação já encontrado (id: 12 )
já tinha
	Verificando: Marcha entity
Marcha já encontrado (id: 6 )
já tinha
	Verificando: rádio cognitivo entity

Relações:




### Passo 6: Recuperar o trecho submetido aos modelos (IDEA-C2-Model - NER e RE), os resultados das inferências (dim_ents e dim_rels) e salvar, respectivamente, nos arquivos de saída: trecho.txt, terms.txt e relations.txt

In [ ]:
# Recuperar as entidades e relações que o modelo conseguiu inferir do texto e salvar na pasta
# em que o código consegue processar e anotar com as entidades e relações que o modelo achou

NOME_ARQUIVO_OUTPUT_TERMS = "/content/drive/MyDrive/Doutorado/IDEA/Código-fonte/docs/terms.txt"
NOME_ARQUIVO_OUTPUT_TERMS2 = "/content/drive/MyDrive/Doutorado/IDEA/Código-fonte/docs/terms2.txt"
NOME_ARQUIVO_OUTPUT_TRECHO = "/content/drive/MyDrive/Doutorado/IDEA/Código-fonte/docs/trecho.txt"

# Processar as relações encontradas e gerar o arquivo entidades.n3
ent_n3=[]

if len(ents) > 0:
  for ent in ents:
    aux = ent[1]
    ent_n3.append(aux)
print(ent_n3)

rels_n3=[]
print(rels)

#Processar as relações encontradas e gerar o arquivo relacoes.n3
if len(rels)>0:
    for rels in rels:
      aux2 = rels[1] + ";" + rels[2]+ ";" + rels[3]
      rels_n3.append(aux2)
#print(dim_rels)

saida = pd.DataFrame(ent_n3)
saida.to_csv(NOME_ARQUIVO_OUTPUT_TERMS, header=None, sep=';', index=False)
saida.head()

saida2 = pd.DataFrame(rels_n3)
saida2.to_csv(NOME_ARQUIVO_OUTPUT_TERMS2, header=None, sep=';', index=False)
saida2.head()

saida3 = pd.DataFrame(text)
saida3.to_csv(NOME_ARQUIVO_OUTPUT_TRECHO, header=None, sep=';', index=False)
saida3.head()

['Operação Militar', 'autoridade', 'guerra', 'Operação Ofensiva', 'Preparação', 'comandante', 'Marcha', 'Ataque', 'ação', 'Perseguição', 'ataque', 'inimigo', 'operação', 'Controle de Tráfego', 'Reconhecimento', 'rádio cognitivo']
[]


,0
0,Uma Operação Militar (Military OPeration) poss...


In [ ]:
print(ents)

[[0, 'Operação Militar', 'entity'], [1, 'autoridade', 'entity'], [2, 'guerra', 'entity'], [3, 'Operação Ofensiva', 'entity'], [4, 'Preparação', 'entity'], [5, 'comandante', 'entity'], [6, 'Marcha', 'entity'], [7, 'Ataque', 'entity'], [8, 'ação', 'entity'], [9, 'Perseguição', 'entity'], [10, 'ataque', 'entity'], [11, 'inimigo', 'entity'], [12, 'operação', 'entity'], [13, 'Controle de Tráfego', 'entity'], [14, 'Reconhecimento', 'entity'], [17, 'rádio cognitivo', 'entity']]


In [ ]:
# Recuperar as entidades e relações que o modelo conseguiu inferir do texto e salvar na pasta
# em que o código consegue processar e anotar com as entidades e relações que o modelo achou

NOME_ARQUIVO_OUTPUT_TERMS = "/content/drive/MyDrive/Doutorado/IDEA/Código-fonte/docs/terms_exp4.txt"
NOME_ARQUIVO_OUTPUT_TERMS2 = "/content/drive/MyDrive/Doutorado/IDEA/Código-fonte/docs/terms2_exp4.txt"
NOME_ARQUIVO_OUTPUT_TRECHO = "/content/drive/MyDrive/Doutorado/IDEA/Código-fonte/docs/trecho_exp4.txt"

# Processar as relações encontradas e gerar o arquivo entidades.n3
ent_n3=[]

if len(dim_ents) > 0:
  for ent in dim_ents:
    aux = ent[1]
    ent_n3.append(aux)
print(ent_n3)

rels_n3=[]
print(dim_rels)

#Processar as relações encontradas e gerar o arquivo relacoes.n3
if len(dim_rels)>0:
    for rels in dim_rels:
      aux2 = rels[1] + ";" + rels[2]+ ";" + rels[3]
      rels_n3.append(aux2)
print(dim_rels)

saida = pd.DataFrame(ent_n3)
saida.to_csv(NOME_ARQUIVO_OUTPUT_TERMS, header=None, sep=';', index=False)
saida.head()

saida2 = pd.DataFrame(rels_n3)
saida2.to_csv(NOME_ARQUIVO_OUTPUT_TERMS2, header=None, sep=';', index=False)
saida2.head()

saida3 = pd.DataFrame(text)
saida3.to_csv(NOME_ARQUIVO_OUTPUT_TRECHO, header=None, sep=';', index=False)
saida3.head()


['Operação Militar', 'Military', 'OPeration', 'Operações Básicas e Complementares', 'Basic and Complentary Operations', 'Operação Ofensiva', 'Offensive Operation', 'Operações Básicas', 'Reunião de Preparação', 'Assembly Area', 'Marcha para o Combate', 'Movement to Contact', 'Ataque Coordenado', 'Organized Offensive', 'Perseguição', 'Exploitation and Follow Up', 'Controle de Tráfego', 'Traffic Control', 'Reconhecimento', 'Initial Reconnaissance', 'C2']
[]
[]


,0
0,Uma Operação Militar (Military OPeration) poss...


In [ ]:
def preparar_ents(text):
  text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
  text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
  text = re.sub(r"[^a-zA-Z.!?']", ' ', text)
  text = re.sub(r" +", ' ', text)
  # text = re.sub(".", "_", text)
  text = re.sub("/", " ", text)
  text = re.sub("[()]/", " ", text)
  text = re.sub(" ", "_", text)
  text = text.lower()
  return text

In [ ]:
import IDEA.singularize
#plurals = ['canetas', 'testes', 'letras']

singles = [preparar_ents(IDEA.singularize.singularize(ents[1])) for ents in ents]
#print(singles)
i=0
j=i+1
lst_sparql=[]
#dim_rels.append([id_rel_cont, e.text, label,b.text])
for i in range(len(singles)):
  for j in range(len(singles)):
    if singles[i] != singles[j]:
      #print(singles[i], singles[j])
      #lst_sparql.append("SELECT * WHERE {?s1 ?p1 ?o1 . ?o1 ?p2 ?s2. FILTER(?s1=cnt:" + singles[i] + " && ?s2=cnt:" + singles[j] + ")}")

      query = "CONSTRUCT {?s1 ?p1 ?o1. ?s2 ?p2 ?o2. ?s3 ?p3 ?o3. ?s4 ?p4 ?o4} "
      query = query + "WHERE {{SELECT * WHERE {?s1 ?p1 ?o1. FILTER(?s1=cnt:" + singles[i] + " )}} "
      query = query + "{SELECT * WHERE {?s2 ?p2 ?o2. FILTER(?o2=cnt:" + singles[i] + ")}} "
      query = query + "{SELECT * WHERE {?s3 ?p3 ?o3. FILTER(?s3=cnt:" + singles[j] + ")}} "
      query = query + "{SELECT * WHERE {?s4 ?p4 ?o4. FILTER(?o4=cnt:" + singles[j] + ")}} "
      query = query + "FILTER(?o2=?s1 && ?o4=?s3 && ?s2=?s4 && ?o1=?o3)}"
      lst_sparql.append(query)

for i in range(len(lst_sparql)):
  print(lst_sparql[i])

CONSTRUCT {?s1 ?p1 ?o1. ?s2 ?p2 ?o2. ?s3 ?p3 ?o3. ?s4 ?p4 ?o4} WHERE {{SELECT * WHERE {?s1 ?p1 ?o1. FILTER(?s1=cnt:operacao_militar )}} {SELECT * WHERE {?s2 ?p2 ?o2. FILTER(?o2=cnt:operacao_militar)}} {SELECT * WHERE {?s3 ?p3 ?o3. FILTER(?s3=cnt:autoridade)}} {SELECT * WHERE {?s4 ?p4 ?o4. FILTER(?o4=cnt:autoridade)}} FILTER(?o2=?s1 && ?o4=?s3 && ?s2=?s4 && ?o1=?o3)}
CONSTRUCT {?s1 ?p1 ?o1. ?s2 ?p2 ?o2. ?s3 ?p3 ?o3. ?s4 ?p4 ?o4} WHERE {{SELECT * WHERE {?s1 ?p1 ?o1. FILTER(?s1=cnt:operacao_militar )}} {SELECT * WHERE {?s2 ?p2 ?o2. FILTER(?o2=cnt:operacao_militar)}} {SELECT * WHERE {?s3 ?p3 ?o3. FILTER(?s3=cnt:guerra)}} {SELECT * WHERE {?s4 ?p4 ?o4. FILTER(?o4=cnt:guerra)}} FILTER(?o2=?s1 && ?o4=?s3 && ?s2=?s4 && ?o1=?o3)}
CONSTRUCT {?s1 ?p1 ?o1. ?s2 ?p2 ?o2. ?s3 ?p3 ?o3. ?s4 ?p4 ?o4} WHERE {{SELECT * WHERE {?s1 ?p1 ?o1. FILTER(?s1=cnt:operacao_militar )}} {SELECT * WHERE {?s2 ?p2 ?o2. FILTER(?o2=cnt:operacao_militar)}} {SELECT * WHERE {?s3 ?p3 ?o3. FILTER(?s3=cnt:operacao_ofensiva)}} {SE